# Raspberry Pi Cameraのデータを可視化するサーバを構築する

## 概要

SINETStreamを利用してRaspberry Piのカメラ画像をKafkaブローカに送信するコンテナを[GitHub](https://github.com/nii-gakunin-cloud/sinetstream-demo/tree/main/VideoStreaming/Sensor/docker)で公開しています。このコンテナから送信されたカメラ画像を可視化するサーバの構築手順を示します。

### システム構成

システム構成を次図に示します。`server`と示されている枠内がこのnotebookで構築する対象になります。

![システム構成](img/02-01-components.png)
<!--
```mermaid
flowchart LR
  subgraph S1[server]
    subgraph K["Apache Kafka"]
      TI(["camera topic"])
      TO(["object list topic"])
    end
    S3[("Object Storage\nMinIO")]
    DB[("PostgreSQL")]
    App["Hasura\nGraphQL Engine"]
    TG["Thumbnail\nGenerator"]
    P["NGINX"]

    TI-..->|"timestamp, etag"|DB----App-..->|"object name,\ntimestamp"|P
    TI==>|"image"|S3==>|image|P
    S3-.->|"object name,\netag"|DB-.->|"object name"|TO-.->TG-.->S3
  end
  subgraph R["Raspberry Pi"]
    C(["Camera"])
    CC["PiCamera\nContainer"]
    C---CC-..->TI
  end
  P-..->W["Web Browser"]
```
-->

サーバを構成するコンポーネントの簡単な説明を以下に記します。

* [Apache Kafka](https://kafka.apache.org/)
  * 分散型イベントストリーミングプラットフォーム
  * コンポーネント間で統一的なインターフェースによるメッセージ処理を行うために利用する
* [MinIO](https://min.io/)
  * S3互換のオブジェクトストレージ
  * カメラ画像の保存先として利用する
* [PostgreSQL](https://www.postgresql.org/)
  * リレーショナルデータベース
  * カメラ画像メタデータなどの保管先として利用する
* [Hasura](https://hasura.io/)
  * GraphQLエンジン
  * データベースに保管されているカメラ画像のメタデータなどを取得、検索するためのインタフェースとして利用する
* [NGINX](https://nginx.org/)
  * Webサーバ
  * カメラ画像データを可視化するWebアプリを配信する
  
また、コンポーネント間でのデータを受け渡すために以下のものを利用します。

* [Kafka Connect](https://kafka.apache.org/documentation/#connect)
  * Kafkaブローカと他コンポーネントの間でデータ転送を行うための枠組み
  * Kafkaブローカとデータをデータベース、オブジェクトストレージの間でデータを転送するために利用する

### 前提条件

サーバを構築、実行するために必要となる前提条件を示します。

* docker, docker compose v2
  * サーバを構成するコンテナを実行するために必要となる
* Python, [jinja2 cli](https://github.com/mattrobenolt/jinja2-cli)
  * サーバを構成する各サービスの設定ファイルなどを生成するために必要となる

前提条件を満たしていることを確認します。以下の４つのセルを実行してエラーにならないことを確認して下さい。

In [ ]:
docker version

In [ ]:
docker compose version

In [ ]:
python3 --version

In [ ]:
jinja2 --version

## パラメータの指定

サーバを構築するために必要となるパラメータを指定します。

### 配置場所 

サーバを構成する資材を配置するディレクトリを次のセルに指定してください。

In [ ]:
# (例)
# target_dir=$HOME/srv/sensor-viewer
# target_dir=/srv/sensor-viewer

target_dir=

資材を配置するディレクトリを作成します。

In [ ]:
mkdir -p $target_dir

### サーバ構成

サーバ構成を指定する設定ファイル`00-config.yml`に、このNotebookで構築するサーバ構成名`picamera`を追加します。

`00-config.yml`の内容を更新するために次のセルを実行してください。

In [ ]:
mkdir -p ${target_dir}/params
touch ${target_dir}/params/.vars_config.yml
[ ! -f ${target_dir}/params/00-config.yml ] || \
cp ${target_dir}/params/00-config.yml ${target_dir}/params/.vars_config.yml
jinja2 \
    -D new_target=picamera \
    -o ${target_dir}/params/00-config.yml \
    files/template/config/00-config.yml.j2 \
    ${target_dir}/params/.vars_config.yml

更新された設定ファイルの内容を表示します。`target`に`picamera`が追加されたことを確認してください。

In [ ]:
cat ${target_dir}/params/00-config.yml

### PiCamera Container

Raspberry Piのカメラデータを送信するPiCamera Containerに関するパラメータを指定します。

カメラデータの送信先となるKafkaのトピック名を指定してください。トピック名は英数字または`-`, `_`のみで構成された文字列として下さい。また複数のRaspberry Piからカメラ画像を送信する場合は、それぞれの送信先となるトピック名を`,`で繋げて指定してください。

In [ ]:
# (例)
# picamera_topics=image-sinetstream-picamera
# picamera_topics=image-sinetstream-picamera1,image-sinetstream-picamera2,image-sinetstream-picamera3

picamera_topics=image-sinetstream-picamera

指定されたパラメータをファイルに保存します。

In [ ]:
jinja2 \
    -D picamera_topics="${picamera_topics:?ERROR: データ送信先となるトピック名が指定されていません}" \
    -o ${target_dir}/params/01-picamera.yml  \
    files/template/config/01-picamera.yml.j2

ファイルの内容を表示します。内容を確認して下さい。

In [ ]:
cat ${target_dir}/params/01-picamera.yml

### Kafka

カメラデータの送信先となるKafkaブローカに関するパラメータを指定します。

Kafkaブローカに関するパラメータが既に設定済みであるかを確認します。次のセルを実行して出力結果が表示された場合は既にKafkaブローカに関するパラメータを設定済みです。この節をスキップして次節に進んでください。

In [ ]:
[ ! -f ${target_dir}/params/01-kafka.yml ] || \
cat ${target_dir}/params/01-kafka.yml

Kafkaブローカのポート番号を指定して下さい。

In [ ]:
# (例)
# kafka_port=9092

kafka_port=9092

Kafkaの外部公開ホスト名を指定してください。Raspberry PiからKafkaブローカにアクセスするときは、ここで指定したホスト名（またはIPアドレス）でアクセス出来るように設定する必要があります。

In [ ]:
# (例)
# kafka_host=kafka.example.org
# kafka_host=192.168.10.100

kafka_hostname=

kafkaのクラスタIDを生成します。

In [ ]:
cluster_id=$(docker run -q --rm apache/kafka:latest /opt/kafka/bin/kafka-storage.sh random-uuid)

指定されたパラメータをファイルに保存します。既に他のnotebookでKafkaに関するパラメータが保存されている場合はその値が優先されます。

In [ ]:
[ -f ${target_dir}/params/01-kafka.yml ] || \
cat > ${target_dir}/params/01-kafka.yml <<EOF
kafka:
  port: ${kafka_port}
  hostname: ${kafka_hostname}
EOF
[ -f ${target_dir}/params/01-kraft.yml ] || \
cat > ${target_dir}/params/01-kraft.yml <<EOF
kraft:
  cluster_id: ${cluster_id}
EOF

ファイルの内容を表示します。内容を確認して下さい。

In [ ]:
cat ${target_dir}/params/01-kafka.yml
cat ${target_dir}/params/01-kraft.yml

### NGINX

送信されたカメラデータを表示するwebサーバ(NGINX)に関するパラメータを指定します。

NGINXに関するパラメータが既に設定済みであるかを確認します。次のセルを実行して出力結果が表示された場合は既にNGINXに関するパラメータを設定済みです。この節をスキップして次節に進んでください。

In [ ]:
[ ! -f ${target_dir}/params/01-www.yml ] || \
cat ${target_dir}/params/01-www.yml

webサーバのプロトコルを指定します。`http`または`https`のどちらかの値を指定して下さい。

In [ ]:
# (例)
# www_protocol=http
# www_protocol=https

www_protocol=

#### サーバ証明書などの指定

webサーバをhttpsで公開する場合はサーバ証明書と秘密鍵などを指定する必要があります。

> `www_protocol`の値に`http`を指定した場合は、この節をスキップして次の「パラメータの保存」からの手順を進めて下さい。

webサーバのホスト名を指定してください。サーバ証明書の内容と一致するホスト名を指定してください。

In [ ]:
# (例)
# hostname=www.example.org

hostname=

サーバ証明書のパスを指定してください。

In [ ]:
# (例)
# cert_file_path=certs/server.crt

cert_file_path=

サーバ証明書の秘密鍵のパスを指定してください。

In [ ]:
# (例)
# cert_key_path=certs/server.key

cert_key_path=

#### パラメータの保存

指定されたパラメータをファイルに保存します。

既に他のnotebookでwebサーバに関するパラメータが保存されている場合はその値が優先され、このnotebookで指定した値は保存されません。

In [ ]:
[ -f ${target_dir}/params/01-www.yml ] || \
cat > ${target_dir}/params/01-www.yml <<EOF
www:
  hostname: ${hostname:-localhost}
  protocol: $www_protocol
EOF

ファイルの内容を表示します。内容を確認して下さい。

In [ ]:
cat ${target_dir}/params/01-www.yml

### MinIO

カメラ画像を保存するオブジェクトストレージ(MinIO)に関するパラメータを指定します。

MinIOに関するパラメータが既に設定済みであるかを確認します。次のセルを実行して出力結果が表示された場合は既にMinIOに関するパラメータを設定済みです。この節をスキップして次節に進んでください。

In [ ]:
if [ -f ${target_dir}/params/01-minio.yml ]; then
    cat ${target_dir}/params/01-minio.yml
    MINIO_ROOT_USER=$(grep user ${target_dir}/params/01-minio.yml | awk '{print $2}')
    MINIO_ROOT_PASSWORD=$(grep password ${target_dir}/params/01-minio.yml | awk '{print $2}')
    bucket_name=$(grep bucket ${target_dir}/params/01-minio.yml | awk '{print $2}')
fi

オブジェクトストレージの管理者として登録するユーザ名を指定してください。

In [ ]:
# (例)
# MINIO_ROOT_USER=myminioadmin

MINIO_ROOT_USER=

オブジェクトストレージ管理者のパスワードを指定してください。

In [ ]:
# (例)
# MINIO_ROOT_PASSWORD=minio-secret-key-change-me

MINIO_ROOT_PASSWORD=

カメラ画像の保存先となるバケット名を指定してください。

In [ ]:
# (例)
# bucket_name=camera

bucket_name=camera

各パラメータに値が設定されていることを確認します。次のセルを実行してエラーにならないことを確認して下さい。

In [ ]:
: ${MINIO_ROOT_USER:?ERROR: 管理者ユーザ名が指定されていません}
: ${MINIO_ROOT_PASSWORD:?ERROR: 管理者のパスワードが指定されていません}
: ${bucket_name:?ERROR: バケット名が指定されていません}

オブジェクトストレージに関するパラメータを保存します。

In [ ]:
[ -f ${target_dir}/params/01-minio.yml ] || \
cat > ${target_dir}/params/01-minio.yml <<EOF
minio:
  root:
    user: $MINIO_ROOT_USER
    password: $MINIO_ROOT_PASSWORD
  bucket: $bucket_name
EOF

ファイルの内容を表示します。内容を確認して下さい。

In [ ]:
cat ${target_dir}/params/01-minio.yml

### PostgreSQL
カメラ画像のメタデータなどを保存するデータベースのパラメータを指定します。

PostgreSQLに関するパラメータが既に設定済みであるかを確認します。次のセルを実行して出力結果が表示された場合は既にPostgreSQLに関するパラメータを設定済みです。この節をスキップして次節に進んでください。

In [ ]:
if [ -f ${target_dir}/params/01-postgres.yml ]; then
    cat ${target_dir}/params/01-postgres.yml
    POSTGRES_DB=$(grep database ${target_dir}/params/01-postgres.yml | awk '{print $2}')
    POSTGRES_USER=$(grep user ${target_dir}/params/01-postgres.yml | awk '{print $2}')
    POSTGRES_PASSWORD=$(grep password ${target_dir}/params/01-postgres.yml | awk '{print $2}')
fi

データベース名を指定してください。

In [ ]:
# (例)
# POSTGRES_DB=sensor

POSTGRES_DB=sensor

ユーザ名を指定してください。

In [ ]:
# (例)
# POSTGRES_USER=sensor

POSTGRES_USER=sensor

パスワードを指定してください。

In [ ]:
# (例)
# POSTGRES_PASSWORD=db-pass-00

POSTGRES_PASSWORD=

各パラメータに値が設定されていることを確認します。次のセルを実行してエラーにならないことを確認して下さい。

In [ ]:
: ${POSTGRES_DB:?ERROR: データベース名が指定されていません}
: ${POSTGRES_USER:?ERROR: データベースのユーザ名が指定されていません}
: ${POSTGRES_PASSWORD:?ERROR: データベースのパスワードが指定されていません}

データベース(PostgreSQL)に関するパラメータを保存します。

In [ ]:
[ -f ${target_dir}/params/01-postgres.yml ] || \
cat > ${target_dir}/params/01-postgres.yml <<EOF
postgres:
  database: ${POSTGRES_DB}
  user: ${POSTGRES_USER}
  password: ${POSTGRES_PASSWORD}
  url: postgres://${POSTGRES_USER}:$(
      python -c "import urllib.parse; print(urllib.parse.quote('$POSTGRES_PASSWORD'))"
  )@postgres:5432/${POSTGRES_DB}?sslmode=disable
EOF

ファイルの内容を表示します。内容を確認して下さい。

In [ ]:
cat ${target_dir}/params/01-postgres.yml

## 資材の配置

サーバを構成するための資材の配置を行います。

### NGINX

Webサーバとして利用するNGINXの設定ファイルなどを配置します。

NGINXの設定ファイルを配置します。

In [ ]:
mkdir -p ${target_dir}/conf/nginx

cat ${target_dir}/params/*.yml | \
jinja2 --strict \
    -o ${target_dir}/conf/nginx/default.conf \
    files/template/nginx/default.conf.j2 

配置した設定ファイルの内容を表示します。

In [ ]:
cat ${target_dir}/conf/nginx/default.conf

NGINXをHTTPSで公開する場合に必要となるサーバ証明書と秘密鍵を配置します。サーバ証明書とその秘密鍵を指定している場合のみ、ファイルの配置が行われます。

In [ ]:
mkdir -p ${target_dir}/secrets
[ -z "$cert_file_path" ] || cp $cert_file_path ${target_dir}/secrets/CERT_FILE
[ -z "$cert_key_path" ] || cp $cert_key_path ${target_dir}/secrets/CERT_KEY

### MinIO

オブジェクトストレージを実行するMinIOコンテナの設定ファイルを配置します。

In [ ]:
mkdir -p ${target_dir}/secrets
cat ${target_dir}/params/*.yml | \
jinja2 --strict \
    -o ${target_dir}/secrets/MINIO_CONFIG \
    files/template/minio/MINIO_CONFIG.j2

ファイルの内容を表示します。内容を確認して下さい。

In [ ]:
cat ${target_dir}/secrets/MINIO_CONFIG

### PostgreSQL

データベースに関する設定ファイルなどを配置します。

データベース名、ユーザ名、パスワードなどを記録したファイルを配置します。これらのパラメータは[docker secret](https://docs.docker.com/engine/swarm/secrets/)として管理します。既に設定済みのファイルが存在している場合は既存のファイルが優先され、ファイルへの書き込みは行いません。

In [ ]:
mkdir -p ${target_dir}/secrets

[ -f ${target_dir}/secrets/POSTGRES_DB ] || \
cat > ${target_dir}/secrets/POSTGRES_DB <<EOF
$POSTGRES_DB
EOF

[ -f ${target_dir}/secrets/POSTGRES_USER ] || \
cat > ${target_dir}/secrets/POSTGRES_USER <<EOF
$POSTGRES_USER
EOF

[ -f ${target_dir}/secrets/POSTGRES_PASSWORD ] || \
cat > ${target_dir}/secrets/POSTGRES_PASSWORD <<EOF
$POSTGRES_PASSWORD
EOF

配置したファイルの内容を表示します。

In [ ]:
cat ${target_dir}/secrets/POSTGRES_DB
cat ${target_dir}/secrets/POSTGRES_USER
cat ${target_dir}/secrets/POSTGRES_PASSWORD

データベースの初期投入SQLファイルを配置します。

In [ ]:
mkdir -p ${target_dir}/init/sql

cat ${target_dir}/params/*.yml | \
jinja2 --strict \
    -o ${target_dir}/params/.vars_sql.yml \
    files/template/sql/vars_sql.yml.j2

cat ${target_dir}/params/.vars_sql.yml ${target_dir}/params/00-config.yml | \
jinja2 --strict \
    -o ${target_dir}/init/sql/create_table.sql \
    files/template/sql/create_table.sql.j2

配置したSQLファイルの内容を確認します。次のセルでは配置したファイルの先頭部分のみを表示しています。必要に応じて| headの部分をコメントアウトしてセルを実行して下さい。

In [ ]:
cat ${target_dir}/init/sql/create_table.sql | head

### Hasura

GraphQLサーバとして利用するHasuraの設定ファイルを配置します。

In [ ]:
mkdir -p ${target_dir}/init/hasura

cp -a files/template/hasura/metadata/* \
    ${target_dir}/init/hasura/

jinja2 --strict \
    -o ${target_dir}/init/hasura/databases/sensor-data/tables/tables.yaml \
    files/template/hasura/tables.yaml.j2 \
    ${target_dir}/params/00-config.yml

### docker-compose.yml

サーバを構成するコンテナに関する設定ファイルを配置します。

`docker-compose.yml`を配置します。

In [ ]:
cat ${target_dir}/params/*.yml | \
jinja2 --strict \
    -o ${target_dir}/docker-compose.yml \
    files/template/docker/docker-compose.yml.j2

配置した設定ファイルの内容を表示します。

In [ ]:
cat ${target_dir}/docker-compose.yml

`docker compose`の環境変数を記した`.env`ファイルを作成します。

In [ ]:
cat ${target_dir}/params/*.yml | \
jinja2 --strict \
    -o ${target_dir}/.env \
    -D uid=$(id -u) -D gid=$(id -g) \
    files/template/docker/dot_env.j2

配置したファイルの内容を表示します。

In [ ]:
cat ${target_dir}/.env

必要となるディレクトリを作成します。

In [ ]:
mkdir -p ${target_dir}/data/postgres ${target_dir}/data/minio

## コンテナの起動

サーバを構成するコンテナを起動します。

利用するコンテナイメージを取得します。

In [ ]:
docker compose --project-directory ${target_dir} pull -q

コンテナを起動します。notebook環境でdocker compose upを実行すると処理中の表示が煩雑なため、次のセルでは全ての出力結果を破棄しています。エラーや警告表示を確認する必要がある場合はnotebook環境ではなく、別窓でターミナルなどを開いてdocker compose upコマンドを実行して下さい。

In [ ]:
docker compose --project-directory ${target_dir} up -d --remove-orphans >& /dev/null

コンテナの実行状況を確認します。

In [ ]:
docker compose --project-directory ${target_dir} ps

## 初期設定

起動したコンテナに対して初期設定を行います。

### PostgreSQL

テーブル定義やビュー定義などを記述したSQLファイルを実行します。

In [ ]:
docker compose --project-directory ${target_dir} exec postgres \
    psql -U $POSTGRES_USER -d $POSTGRES_DB \
    -f /docker-entrypoint-initdb.d/020_create_table.sql

確認のためテーブル、ビューなどのリレーションの一覧を表示してみます。

In [ ]:
docker compose --project-directory ${target_dir} exec postgres \
    psql -U $POSTGRES_USER -d $POSTGRES_DB -c "\pset pager off" -c "\d"

テーブル定義の変更を反映するためにgraphqlコンテナを再起動します。

In [ ]:
docker compose --project-directory ${target_dir} restart graphql >& /dev/null

### MinIO

オブジェクトストレージの初期設定スクリプトを実行します。

次のセルを実行すると以下の項目の初期設定が行われます。

* バケットの作成
* アクセス権の設定
* オブジェクトの変更通知設定

In [ ]:
cat > files/setup/minio/.env <<EOF
target_dir=${target_dir}
BUCKET_NAME=${bucket_name}
EOF

docker compose --project-directory files/setup/minio \
    -f files/setup/minio/docker-compose-setup.yml up --quiet-pull

### Kafka Connect

Kafkaブローカと他コンポーネントの間でデータの送受信を処理するKafka Connectの設定を行います。

#### KafkaからMinIO

Raspberry PiからKafkaブローカに送信されたカメラ画像をオブジェクトストレージに保存する設定を行います。

In [ ]:
env ACCESS_KEY=$MINIO_ROOT_USER SECRET_KEY=$MINIO_ROOT_PASSWORD \
files/setup/kafka-connect/setup-sink-minio.sh -t $picamera_topics -b $bucket_name \
    -u http://minio:9000 -D -n sink-minio-picamera

登録状況を確認します。`jq`コマンドが利用できない場合は、次のセルの末尾の`| jq .`の部分をコメントアウトして実行して下さい。

In [ ]:
curl -s http://localhost:8083/connectors/sink-minio-picamera | jq .

#### KafkaからPostgreSQL

Raspberry PiからKafkaブローカに送信されたカメラ画像のタイムスタンプなどのメタデータをデータベースに保存する設定を行います。

In [ ]:
env POSTGRES_DB=$POSTGRES_DB POSTGRES_USER=$POSTGRES_USER POSTGRES_PASSWORD=$POSTGRES_PASSWORD \
files/setup/kafka-connect/setup-sink-psql-etag.sh -t $picamera_topics -n sink-psql-etag

登録状況を確認します。

In [ ]:
curl -s http://localhost:8083/connectors/sink-psql-etag | jq .

#### PostgreSQLからKafka

Webサーバで画像を表示する際の応答時間を短くするために事前に縮小画像をサーバ側で生成しておきます。縮小画像を生成するためにオブジェクトストレージに保存された画像ファイルのリストをデータベースから取得してKafkaブローカを経由して縮小画像生成コンテナに送ります。このための設定を行います。

In [ ]:
env POSTGRES_DB=$POSTGRES_DB POSTGRES_USER=$POSTGRES_USER POSTGRES_PASSWORD=$POSTGRES_PASSWORD \
files/setup/kafka-connect/setup-source-psql.sh -t minio-sinetstream- -n source-psql-minio-event

登録状況を確認します。

In [ ]:
curl -s http://localhost:8083/connectors/source-psql-minio-event | jq

## センサーデータの可視化結果の表示

構築したwebサーバにアクセスして可視化結果を表示してみます。次のセルを実行すると表示されるアドレスにアクセスして下さい。

In [ ]:
echo "${www_protocol}://${hostname:-localhost}"

上のセルの実行結果に表示されたアドレスに初めてアクセスすると下図のような設定画面が表示されます。

> 既に初回設定を済ませている場合は可視化画面が表示されます。

![viewer初期画面](img/viewer-001.png)

設定画面で直接各項目を入力することもできますが、サーバ構築時に登録されたデフォルト設定をダウンロードすることもできます。

デフォルト設定をダウンロードする場合は上図の赤丸で示したアイコンを選択して下さい。次図のような画面が表示されます。ドロップダウンリストによりサーバ側に登録された設定内容を選択することができます。初回設定では、サーバ構築時に登録された`default`という設定内容のみが存在しています。

![viewerダウンロード画面](img/viewer-005.png)

ドロップダウンリストで`default`を選択し、画面下部に表示されている`Apply`ボタンをクリックして下さい。次図に示すようにサーバ側に登録されている内容が設定画面に取り込まれます。

![viewer設定画面](img/viewer-006.png)

設定画面の`name`欄に設定名を入力して画面下部の`Save`ボタンをクリックすることで設定内容がWebブラウザに保存されます。その後、次図のようなカメラ画像を表示する可視化画面となります。

![viewer可視化画面](img/viewer-007.png)

## 送信側の環境構築

カメラ画像を送信するRaspberry Pi側の環境構築手順については[12-setup-raspi-camera.ipynb](../Sensor/RaspberryPi/12-setup-raspi-camera.ipynb)を参照して下さい。